# Graph Databases and Vector Search with Neo4j

In this lab, you will use the Cypher query language to query highly connected data in the graph database Neo4j. Relationships between data entities can be just as important as the data itself, and graph databases are designed to answer questions about the relationships. You will also leverage the graph database to perform vector search: a method of information retrieval where data points are represented as vectors. One can perform a similarity search between data points by comparing their vector representations. 

# Table of Contents

- [ 1 - Introduction and Setup](#1)
  - [ 1.1 - Introduction](#1.1)
  - [ 1.2 - Development Environment](#1.2)
- [ 2 - Basic Operations with Graph Database](#2)
  - [ 2.1 - Introduction to Cypher](#2.1)
  - [ 2.2 - Getting Node Information with Match Statements](#2.2)
    - [ Exercise 1](#ex01)
    - [ Exercise 2](#ex02)
  - [ 2.3 - Exploring Relationships](#2.3)
    - [ Exercise 3](#ex03)
    - [ Exercise 4](#ex04)
  - [ 2.4 - Getting Properties](#2.4)
    - [ Exercise 5](#ex05)
    - [ Exercise 6](#ex06)
  - [ 2.5 - Filtering Statements](#2.5)
    - [ Exercise 7](#ex07)
  - [ 2.6 - Create Statements](#2.6)
    - [ Exercise 8](#ex08)
  - [ 2.7 - Update Operations](#2.7)
    - [ Exercise 9](#ex09)
    - [ Exercise 10](#ex10)
  - [ 2.8 - Delete Statements](#2.8)
    - [ Exercise 11](#ex11)
    - [ Exercise 12](#ex12)
    - [ Exercise 13](#ex13)
- [ 3 - Advanced Queries](#3)
  - [ Exercise 14](#ex14)
  - [ Exercise 15](#ex15)
  - [ Exercise 16](#ex16)
- [ 4 - Upload Files for Grading](#4)
- [ 5 - Vector Search](#5)
  - [ Exercise 17](#ex17)

<a name='1'></a>
## 1 - Introduction and Setup

<a name='1.1'></a>
### 1.1 - Introduction

You are employed by AeroTrack Insights, an organization committed to compiling data on airline routes and devising strategies for enhancing travel efficiency. Your current task involves working with the newly implemented graph database called 'Air Routes', which comprehensively represents a significant portion of the global airline route network. This database encompasses data about airports across several countries and provinces, spanning all continents. Additionally, it includes details regarding the routes connecting these airports, countries, and continents. To begin extracting valuable insights from this database, it is imperative to acquaint yourself with graph databases and the range of queries they enable. 

Relationships between different entities are shown in the following diagram:

![image](./images/air-routes-graph.png)

<a name='1.2'></a>
### 1.2 - Development Environment

1.2.1. Import some packages that you will use during this lab.

In [ ]:
import os 
import json
import subprocess

from dotenv import load_dotenv
from neo4j import GraphDatabase

LAB_PREFIX='de-c3w1a1'

1.2.2. Although you are provided with this notebook to perform your queries and the exercises, you are also encouraged to explore the Neo4j graphical interface and execute the same queries there to better visualize the results. To access the graphical interface, go to your AWS management console and search for **CloudFormation**. You will see two stacks deployed, one associated with your Cloud9 environment (name with prefix `aws-cloud9`) and another named with an alphanumeric ID. Click on the alphanumeric ID stack, and navigate to the **Outputs** tab. Copy the value of the `Neo4jDNSUI` key (including the port at the end of it). Paste it to a new browser window, and use the following credentials to access the database::

* username: `neo4j`
* password: `adminneo4j`

In that graphical interface, you can see the results not only as plain text but also visualize them as graphs. If you want to visualize the result of the queries that you are going to create during this lab, you can always go to this graphical interface and execute them again.

1.2.3. Open the `./src/env` file and replace the placeholder `<Neo4j-DNS-Connection>` with the `Neo4jDNSConnection` value from the CloudFormation Outputs. **Please do not include the colon and the port number there!** As you can see, the port number is set in that file separately. Save changes and run the following cell:

In [ ]:
load_dotenv('./src/env', override=True)

<a name='2'></a>
## 2 - Basic Operations with Graph Database

<a name='2.1'></a>
### 2.1 - Introduction to Cypher

Cypher is a declarative query language designed for expressing queries across graph databases. It provides a concise and intuitive syntax for performing operations such as creating, updating, retrieving, and deleting data within graph structures. It reuses syntax from SQL and mixes it with ASCII elements to represent graph elements. OpenCypher, on the other hand, is an open standard for graph query languages inspired by Cypher. It aims to standardize the Cypher query language across different graph database implementations. Alongside Cypher, Gremlin and SPARQLВ are the most popular Graph Query Languages.

When using Cypher to query your graph database, you need to understand the difference between nodes, relationships, and paths. Let's take a closer look at these components.

#### Nodes

A node is used to capture a data item, usually an entity, like a customer, an order, a product, etc.

- `()`:  This represents a *node*. You did not specify a specific type of node or any properties of that node. It's not relevant to the query.
- `(n)`:  This represents a *node* referred to by the variable **n**. You can refer to this variable in other parts of your query.
- `(n:Airport)`: Nodes can have different types (i.e. they can belong to different classes/categories). You can add a *label* to your node to specify its type. Here you are assigning the variable **n** the nodes with the type **Airport**.
- `(n:Airport {code: 'BOS', desc: 'Boston Logan'})`: A node can have properties, which you can specify with `{}`. Here you are assigning the variable **n** the nodes of type **Airport** that have specific values for the **code** and **desc** properties.
- `n.code`: You can access a specific property using this syntax, in this case, the **code** from the node denoted by **n**.

#### Relationships/Edges

A relationship or edge is used to describe a connection between two nodes.

- `[r]`: This represents a *relationship* referred to by the variable **r**. You can refer to this variable in other parts of your query. 
- `[r:Route]`: Relationships can have different types (i.e. they can belong to different classes/categories). You can add a *label* to your *relationship* to specify its type. Here you are assigning the variable **r** the relationships with the type **Route**.
- `[:Route]`: A *relationship* with the *label* **Route** not referred to by any variable.
- `[r:Route {dist:809}]`: Relationships can have properties, which you can specify with `{}`. Here you are assigning the variable **r** the *relationships* of type **Route** that have specific values for the **dist** property.
- `[r:Route*..4]`: This syntax is used to match a pattern where the relationship **r** with the label **route** can be repeated between 1 to 4 times. In other words, it matches paths where the **route** relationship occurs consecutively at least once and at most four times.

#### Paths
A path is used to capture the graph structure.

- `(a:Airport)-[:Route]-(b:Airport)`: This represents a *path* that describes that node **a** and node **b** are connected by a **Route** relationship.
- `(a:Airport)-[:Route]->(b:Airport)`: A path can be directed. In this case, this represents a path that describes a directed relationship from node **a** to node **b**, but not the other way around. 
- `(a:Airport)<-[:Route]-(b:Airport)`: A path that describes a directed relationship from node **b** to node **a**, but not the other way around. 
- `(a:Airport)-[:Route]-(b:Airport)-[:Route]-(c:Airport)`: A path can chain multiple relationships and any of them can be directional.

You will see more about nodes, relationships and paths in the next exercises while exploring the syntax of the language. 

The **variables** will appear by naming parts of the patterns or a query to reference them. You will see the examples below.


#### Pattern Matching Syntax

In the following table you can find the characters that represent each component in the Cypher language:

|   Cypher Pattern|Description|
|:--|:--|
|`( )`|A node|
|`[ ]`|An edge|
|`-->`|Follow outgoing edges from a node|
|`<--`|Follow incoming edges to a node|
|`--`|Follow edges in either direction|
|`-[]->`|Include the outgoing edges in the query (for example, to check a label or property)|
|`<-[]-`|Include the incoming edges in the query (for example, to check a label or property)|
|`-[]-` |Include edges in either direction in the query|
|`-[]->( )`|The node on the other end of an outgoing edge|
|`<-[]-()`|The node on the other end of an incoming edge|


To find more information about Cypher/OpenCypher, you can visit these resources:
- [Cypher Manual](https://neo4j.com/docs/cypher-manual/current/introduction/)
- [LearnXinYminutes Cypher](https://learnxinyminutes.com/docs/cypher/)

Let's explore and understand the dataset while having some practice with Cypher. 

<a name='2.2'></a>
### 2.2 - Getting Node Information with Match Statements

Start by defining a function to execute queries in the graph database and retrieve the result.

In [ ]:
URI = f"neo4j://{os.getenv('NEO4JHOST')}:{os.getenv('PORT')}"
#В Add your database credentials in the format: (username, password)
AUTH = (os.getenv('USERNAME'), os.getenv('PASSWORD'))

def execute_query(query:str, db_uri:str = URI, auth: tuple[str, str] = AUTH) -> str:
    """Connects to a Neo4j database and sends a query

    Args:
        query (str): Query to be performed in Ne4j graph DB
        db_uri (str): Database URI
        auth (tuple): Tuple of strings with user and password for the DB

    Returns:
        str: Query results
    """

    with GraphDatabase.driver(db_uri, auth=auth) as driver:
        driver.verify_connectivity()
        records, summary, keys = driver.execute_query(query,database_="neo4j",)
        
        return json.dumps([r.data() for r in records], indent=2)

Match statements in Cypher are used to retrieve data from the graph by specifying patterns of nodes and relationships. These patterns define the structure of the data you want to retrieve or manipulate. The `MATCH` statement is used to specify patterns of nodes and relationships to match in the graph. It is the primary way to retrieve data from the graph. The `RETURN` keyword is used to specify what data to include in the query result. It specifies the properties of nodes and relationships to return, as well as any computed values. The `LIMIT` keyword limits the number of returned values. Run each of the following cells to better understand how `MATCH` statements work.

**Match all nodes**: Let's get some nodes, limited only to 20 values. Note that you are using `(n)` to get a *node* referred to the variable **n**, as mentioned in an earlier section.

In [ ]:
query = "MATCH (n) RETURN n LIMIT 20"
records = execute_query(query)
print(records)

**Count all nodes**: As mentioned before, the `RETURN` keyword allows to return not only nodes or relationships, but also computed values. Here, you will count all the nodes available in the dataset with the `count()` function. Also, note that you can use aliases for your results with the `AS` keyword.

In [ ]:
query = "MATCH (n) RETURN count(n) AS count"
records = execute_query(query)
print(records)

Now, let's query which types of nodes represent the air routes network. Nodes can have labels, which allow you to identify the type or category of each node. 

<a name='ex01'></a>
### Exercise 1

**Get node labels and count the number of nodes per label**. This involves using the `labels()` function, which retrieves the labels for a given node `n`, and the `count(*)` function, which counts the number of nodes. Use the `DISTINCT` keyword straight after the `RETURN` keyword to get only the unique labels.

In [ ]:
### START CODE HERE ### (1 line of code)
query = "MATCH (n) RETURN None None(None), None(None)"
### END CODE HERE ###
records = execute_query(query)
print(records)

##### __Expected Output__ 

```json
[
  {
    "labels(n)": [
      "Continent"
    ],
    "count(*)": 7
  },
  {
    "labels(n)": [
      "Airport"
    ],
    "count(*)": 3504
  },
  {
    "labels(n)": [
      "Country"
    ],
    "count(*)": 237
  }
]
```

**Match nodes by label**: You can add the type or category of a node by passing a label. Here you will get only the nodes labelled as `Airport`.

In [ ]:
query = "MATCH (a:Airport) RETURN a LIMIT 10"
records = execute_query(query)
print(records)

<a name='ex02'></a>
### Exercise 2

Complete the code to explore the nodes with the label `Country`. Limit your result to 10 values.

In [ ]:
### START CODE HERE ### (2 lines of code)
query = "MATCH (c:None) RETURN c None None"
records = None(None)
### END CODE HERE ###

print(records)

##### __Expected Output__ 

*Note*: Not all of the output is shown and the order of the nodes can be different.

```json
[
  {
    "c": {
      "code": "AF",
      "id": 3505,
      "type": "country",
      "desc": "Afghanistan"
    }
  },
  {
    "c": {
      "code": "AL",
      "id": 3506,
      "type": "country",
      "desc": "Albania"
    }
  },
...
]
```

<a name='2.3'></a>
### 2.3 - Exploring Relationships

Now explore the **relationships between the nodes**. Execute the following query to return ten rows of a generic path using the path syntax. 

In [ ]:
query = "MATCH (n)-[r:Route]-() RETURN n, r LIMIT 10"
records = execute_query(query)
print(records)

Remember that relationships are defined by using square brackets such as `[r:route]`. In this case, you will see that you are searching for all nodes `()` that are related. To find all the relationships that are directed, you can use the syntax provided in the **Paths** description (see [2.1](#2.1)) in which by using `->` you can indicate a directed relationship.

<a name='ex03'></a>
### Exercise 3

Write a query to **count all relationships**. As you want to count the relationships `r`, complete the code with the `count(r)` function. Name the result as `relationships_count`.

In [ ]:
### START CODE HERE ### (1 line of code)
query = "MATCH ()-[r]->() RETURN None(None) AS None"
### END CODE HERE ###
records = execute_query(query)
print(records)

##### __Expected Output__ 

```json
[
  {
    "relationships_count": 57645
  }
]
```

<a name='ex04'></a>
### Exercise 4

Create a query that returns the **count of relationships per type**. Use the function `TYPE()` passing the relationship `r` and `count(*)` for that purpose. You should add the `DISTINCT` keyword straight after the `RETURN` keyword. 

In [ ]:
### START CODE HERE ### (1 line of code)
query = "MATCH ()-[r]-() RETURN None None(None), None(None)"
### END CODE HERE ###
records = execute_query(query)
print(records)

##### __Expected Output__ 

```json
[
  {
    "TYPE(r)": "Route",
    "count(*)": 101274
  },
  {
    "TYPE(r)": "Contains",
    "count(*)": 14016
  }
]
```

**Match nodes according to relationships (undirected)**: You are going to search the paths that connect two airports. In this case, there is no direction in the relationship as you can travel from one airport to another and vice versa. This is indicated by using a simple hyphen (`-`) between the nodes and the relationship.

In [ ]:
query = """
        MATCH (a:Airport)-[:Route]-(b:Airport)
        RETURN a,b
        LIMIT 10
        """
records = execute_query(query)
print(records)

**Match nodes according to relationships (directed)**: now you're going to search the paths that represent directed relationships. In your dataset, Countries and Continents contain Airports, which indicates a directed relationship as the inverse relationship is not true (for example, Airports do not contain Continents). For that, you will use the syntax `()-[r:Contains]->()` where the `->` indicates the direction of the relation and `Contains` is the label of the relationship. When returning a path, you assign the entire pattern to a variable, as seen below where you'll assign the path to a variable `p`.

In [ ]:
query = "MATCH p=()-[r:Contains]->() RETURN p LIMIT 10"
records = execute_query(query)
print(records)

<a name='2.4'></a>
### 2.4 - Getting Properties

Apart from a label, nodes can have some other properties.

<a name='ex05'></a>
### Exercise 5

Complete the code to **get properties of the nodes**. Use the `properties()` function passing the node variable `a` to get the properties of nodes with the label `Airport`. Limit your result to only 1 node.

In [ ]:
### START CODE HERE ### (1 line of code)
query = "MATCH (None:None) RETURN None(None) LIMIT None"
### END CODE HERE ###
records = execute_query(query)
print(records)

##### __Expected Output__ 

```json
[
  {
    "properties(a)": {
      "region": "US-AK",
      "lon": -149.996002197266,
      "desc": "Anchorage Ted Stevens",
      "elev": 151,
      "code": "ANC",
      "type": "airport",
      "longest": 12400,
      "country": "US",
      "city": "Anchorage",
      "id": 2,
      "runways": 3,
      "icao": "PANC",
      "lat": 61.1744003295898
    }
  }
]
```

Not only can nodes have properties, but relationships can have properties, too!

<a name='ex06'></a>
### Exercise 6

Complete the query to **get properties of the relationship** with type `Route`. Use the `properties()` function passing the relationship `r`. Limit your results to just 1 value.

In [ ]:
### START CODE HERE ### (1 line of code)
query = "MATCH ()-[r:Route]-() RETURN None(None) LIMIT None"
### END CODE HERE ###
records = execute_query(query)
print(records)

##### __Expected Output__ 

```json
[
  {
    "properties(r)": {
      "dist": 1732
    }
  }
]
```

**Match nodes by label and property**:  You can use the properties to filter your results and extract only the nodes with a particular value of a certain property.

In [ ]:
query = "MATCH (a:Airport {code : 'BOS'}) RETURN a"
records = execute_query(query)
print(records)

**Match all relationships and aggregate a property**: Properties of relationships can be used to perform aggregations. For example, let's get the average distance of all routes.

In [ ]:
query = "MATCH ()-[r:Route]->() RETURN avg(r.dist)"
records = execute_query(query)
print(records)

<a name='2.5'></a>
### 2.5 - Filtering Statements

**Match nodes with a `WHERE` clause**: The `WHERE` statement is used to filter the results based on conditions. It allows you to specify conditions that nodes, relationships, or properties must meet to be included in the result. The syntax looks like

```cypher
MATCH <PATTERN>
WHERE <NODE-OR-RELATION>.<PROPERTY> = <VALUE>
RETURN <PATTERN>
```
    

<a name='ex07'></a>
### Exercise 7

Extract the origin airport from routes with a distance larger than 1000 miles from the destination airport. These nodes can be filtered out using the route `r` property `.dist`. Return the nodes `a` limiting the result to only 5 values.

In [ ]:
### START CODE HERE ### (1 line of code)
query = "MATCH (a:Airport)-[r:Route]->(b:Airport) WHERE None.None > 1000 RETURN None LIMIT None"
### END CODE HERE ###

records = execute_query(query)
print(records)

##### __Expected Output__ 

*Note*: Not all of the output is shown and the order of the nodes can be different.

```json
[
  {
    "a": {
      "country": "US",
      "longest": 11489,
      "code": "PHX",
      "city": "Phoenix",
      "lon": -112.012001037598,
      "type": "airport",
      "elev": 1135,
      "icao": "KPHX",
      "id": 20,
      "runways": 3,
      "region": "US-AZ",
      "lat": 33.4342994689941,
      "desc": "Phoenix Sky Harbor International Airport"
    }
  },
...
]
```

<a name='2.6'></a>
### 2.6 - Create Statements

Let's perform some **creation operations**. The `CREATE` statement is used to create new nodes and relationships in the graph. It allows you to specify the structure and properties of the new elements. To create a new node with some properties and return the node you can use a syntax like:

```cypher
CREATE (<VARIABLE>:<LABEL> {<PROPERTY> : <VALUE>, ..., <PROPERTY> : <VALUE>})
RETURN <VARIABLE>
```

In the next two cells you will add the following two airports:

1. **Airport 1:**
  * Country: "US"
  * Longest: 1008
  * Code: "CLR"
  * City: "Calipatria"
  * Longitud: -115.521389
  * Type: "airport"
  * Elevation: -182
  * ICAO Code: "KCLR"
  * ID: 3800
  * Region: "US-CA"
  * Runaways: 1
  * Latitude: 33.131389
  * Description: "Cliff Hatfield Memorial Airport"


2. **Airport 2:**
  * Country: "US"
  * Longest: 803
  * Code: "BWC"
  * City: "Brawley"
  * Longitud: -115.516944
  * Type: "airport"
  * Elevation: -100
  * ICAO Code: "KBWC"
  * ID: 3801
  * Region: "US-CA"
  * Runaways: 1
  * Latitude: 32.993056
  * Description: "Brawley Municipal Airport"

In [ ]:
query = "CREATE (a:Airport {country : 'US', longest : 1008, code: 'CLR', city: 'Calipatria', lon: -115.521389, type: 'airport', elev: -182, icao: 'KCLR', id: 3800, region: 'US-CA', runaways: 1, lat: 33.131389, desc: 'Cliff Hatfield Memorial Airport' }) RETURN a"
records = execute_query(query)
print(records)

In [ ]:
query = "CREATE (a:Airport {country : 'US', longest : 803, code: 'BWC', city: 'Brawley', lon: -115.516944, type: 'airport', elev: -100, icao: 'KBWC', id: 3801, region: 'US-CA', runaways: 1, lat: 32.993056, desc: 'Brawley Municipal Airport' }) RETURN a"
records = execute_query(query)
print(records)

To create a new relationship between the nodes, you can use `MATCH` statement to search the nodes you want to join according to a property and then the `CREATE` statement to introduce the relationship, such as:

```cypher
MATCH (<NODE1>:<TYPE> {<PROPERTY>: <VALUE>}), (<NODE2>:<TYPE> {<PROPERTY>: <VALUE2>})
CREATE (<NODE1>)-[:<RELATIONSHIP> {<PROPERTY>: <VALUE>}]->(<NODE2>)
```

Take into account that at creation time, relationships between nodes must be directed in Cypher. Despite that, you can always query it as an undirected relationship. 

<a name='ex08'></a>
### Exercise 8

Use the two nodes that you just created using the nodes' `code` properties `'CLR'` and `'BWC'`. Set the distance property `dist` as 12 (there are around 12 miles between the two airports). Return the created relationship.

In [ ]:
### START CODE HERE ### (1 line of code)
query = "MATCH (a:Airport {None: 'None'}), (b:Airport {None: 'None'}) CREATE (a)-[r:Route {None: None}]->(b) RETURN None"
### END CODE HERE ### 
records = execute_query(query)
print(records)

##### __Expected Output__ 

```json
[
  {
    "r": [
      {},
      "Route",
      {}
    ]
  }
]
```

Now that you created the nodes and the relationship, query the path with all the destinations associated with the `'CLR'` origin airport.

In [ ]:
query = """
        MATCH p=(a:Airport {code: 'CLR'})-[r]->(n)        
        RETURN p
        """
records = execute_query(query)
print(records)

##### __Expected Output__ 

```json
[
  {
    "p": [
      {
        "country": "US",
        "longest": 1008,
        "code": "CLR",
        "city": "Calipatria",
        "lon": -115.521389,
        "type": "airport",
        "elev": -182,
        "runaways": 1,
        "icao": "KCLR",
        "id": 3800,
        "region": "US-CA",
        "lat": 33.131389,
        "desc": "Cliff Hatfield Memorial Airport"
      },
      "Route",
      {
        "country": "US",
        "longest": 803,
        "code": "BWC",
        "city": "Brawley",
        "lon": -115.516944,
        "type": "airport",
        "elev": -100,
        "runaways": 1,
        "icao": "KBWC",
        "id": 3801,
        "region": "US-CA",
        "lat": 32.993056,
        "desc": "Brawley Municipal Airport"
      }
    ]
  }
]
```

<a name='2.7'></a>
### 2.7 - Update Operations

You can use `MATCH`, `WHERE` and `SET` statements to update node properties, using a syntax like:

```cypher
MATCH (<NODE>:<TYPE>)
WHERE <NODE>.<CONDITIONAL-PROPERTY> = <CONDITIONAL-VALUE>
SET <NODE>.<PROPERTY-TO-UPDATE> = <NEW-VALUE>
```

With `MATCH` and `WHERE` you will select and filter a node according to a particular condition, while the `SET` statement is used to update the values. 

<a name='ex09'></a>
### Exercise 9

Complete the query to update the airport with code `'BWC'`. Its elevation (property `elev`) is actually -128 ft instead of the -100 ft value that has been inserted previously.

In [ ]:
### START CODE HERE ### (1 line of code)
query = "MATCH (a:None) WHERE None.None = 'None' SET None.None = -128 RETURN a"
### END CODE HERE ###
records = execute_query(query)
print(records)

##### __Expected Output__ 

```json
[
  {
    "a": {
      "country": "US",
      "longest": 803,
      "code": "BWC",
      "city": "Brawley",
      "lon": -115.516944,
      "type": "airport",
      "elev": -128,
      "runaways": 1,
      "icao": "KBWC",
      "id": 3801,
      "region": "US-CA",
      "lat": 32.993056,
      "desc": "Brawley Municipal Airport"
    }
  }
]
```

<a name='ex10'></a>
### Exercise 10

Now, query the airport with code `'BWC'` and return the elevation to check that the value has been updated.

In [ ]:
### START CODE HERE ### (1 line of code)
query = "MATCH (a:None {None: 'None'}) RETURN None.None"
### END CODE HERE ### 
records = execute_query(query)
print(records)

##### __Expected Output__ 

```json
[
  {
    "a.elev": -128
  }
]
```

<a name='2.8'></a>
### 2.8 - Delete Statements

As an important part of the CRUD operations, the `DELETE` statement is used to delete nodes and relationships from the graph. It allows you to specify the elements to delete based on patterns or conditions. 

With the deletion of a specific node, you should also delete the associated relationships. Here's the syntax for deletion:

```cypher
MATCH (<NODE>:<TYPE>)-[r]-()
WHERE <NODE>.<PROPERTY> = <VALUE>
DELETE r, <NODE>

```

In the next exercises, you will delete the last two nodes that you inserted.

<a name='ex11'></a>
### Exercise 11

Delete the airport node with the code `'CLR'` and the relationship associated with it.

In [ ]:
### START CODE HERE ### (1 line of code)
query = "MATCH (a:None)-[r]-() WHERE None.None = 'None' DELETE None, None"
### END CODE HERE ### 
records = execute_query(query)
print(records)

##### __Expected Output__ 

```json
[]
```

<a name='ex12'></a>
### Exercise 12

Delete the second airport node with the code `'BWC'`.

In [ ]:
### START CODE HERE ### (1 line of code)
query = "MATCH (a:None) WHERE None.None = 'None' DELETE None"
### END CODE HERE ### 
records = execute_query(query)
print(records)

##### __Expected Output__ 

```json
[]
```

<a name='ex13'></a>
### Exercise 13

Complete the query to search for the two deleted nodes (codes `'CLR'` and `'BWC'`). The expected output is an empty list.

In [ ]:
### START CODE HERE ### (1 line of code)
query = "MATCH (a:None {None: 'None'}), (b:None {None: 'None'}) RETURN None, None"
### END CODE HERE ### 
records = execute_query(query)
print(records)

##### __Expected Output__ 

```json
[]
```

<a name='3'></a>
## 3 - Advanced Queries

In the previous section, you have learned the basic syntax for the CRUD operations in Cypher. In the following exercises, you will create some more advanced queries to search for complex paths and patterns in your data.

<a name='ex14'></a>
### Exercise 14

Find **the number of all direct routes** from LaGuardia Airport in New York (code `'LGA'`). You will need to use the syntax for a path between two nodes and the function `count()` applied to the `r`. Add the `DISTINCT` keyword before that. Name your result as `count_routes`.

In [ ]:
### START CODE HERE ### (1 line of code)
query = "MATCH (origin:Airport {None: 'None'})-[r:None]->(dest:Airport) RETURN None None(None) AS None"
### END CODE HERE ###
records = execute_query(query)
print(records)

##### __Expected Output__ 

```json
[
  {
    "count_routes": 81
  }
]
```

In the next exercise, you will use **WITH** clause which allows you to chain query parts together, carrying over variables or introducing new ones for further operations. As it has been noted above, variables do not get passed to the subsequent parts of the query, unless they are included in the WITH clause. This feature is particularly useful for performing aggregations, filtering results, or managing complex queries by breaking them down into simpler, manageable parts.

<a name='ex15'></a>
### Exercise 15

Show 10 airports located in the United States (`country` property equal to `US`) that only have one route. You will need to use the function `count()` and `WHERE` statement based on the result of that. The WITH clause will be used to count the routes for each airport and carry over the results for filtering.

In [ ]:
### START CODE HERE ### (1 line of code)
query = "MATCH (a:Airport)-[r:Route]->(b:Airport) WITH a, None(r) AS count_routes WHERE None=1 and None.None = 'None' RETURN a LIMIT 10"
### END CODE HERE ###
records = execute_query(query)
print(records)

##### __Expected Output__ 

*Note*: Not all of the output is shown and the order of the nodes can be different.

```json
[
  {
    "a": {
      "country": "US",
      "longest": 6601,
      "code": "TXK",
      "city": "Texarkana",
      "lon": -93.9909973144531,
      "type": "airport",
      "elev": 390,
      "icao": "KTXK",
      "id": 300,
      "runways": 2,
      "region": "US-AR",
      "lat": 33.4537010192871,
      "desc": "Texarkana Regional Webb Field"
    }
  },
...
]
```

<a name='ex16'></a>
### Exercise 16

Find the possible routes that use only one intermediary airport starting from Columbia Regional Airport (`'COU'`) and ending in Miami International Airport (`'MIA'`). Remember the syntax to define how many relationships are needed in a path.

The syntax `[:Route*..2]` is used to capture paths with up to 2 relationships, meaning direct flights and those with one intermediary airport.

In [ ]:
### START CODE HERE ### (1 line of code)
query = "MATCH paths=(origin:Airport {code: 'None'})-[:Route*..2]->(dest:Airport{code: 'None'}) RETURN None"
### END CODE HERE ###
records = execute_query(query)
print(records)

##### __Expected Output__ 

*Note*: Not all of the output is shown and the order of the nodes can be different.

```json
[
{
    "paths": [
      {
        "country": "US",
        "longest": 6501,
        "code": "COU",
        "city": "Columbia",
        "lon": -92.219596862793,
        "type": "airport",
        "elev": 889,
        "icao": "KCOU",
        "id": 377,
        "runways": 2,
        "region": "US-MO",
        "lat": 38.8180999755859,
        "desc": "Columbia Regional Airport"
      },
      "Route",
      {
        "country": "US",
        "longest": 13000,
        "code": "ORD",
        "city": "Chicago",
        "lon": -87.90480042,
        "type": "airport",
        "elev": 672,
        "icao": "KORD",
        "id": 18,
        "runways": 7,
        "region": "US-IL",
        "lat": 41.97859955,
        "desc": "Chicago O'Hare International Airport"
      },
      "Route",
      {
        "country": "US",
        "longest": 13016,
        "code": "MIA",
        "city": "Miami",
        "lon": -80.2906036376953,
        "type": "airport",
        "elev": 8,
        "icao": "KMIA",
        "id": 16,
        "runways": 4,
        "region": "US-FL",
        "lat": 25.7931995391846,
        "desc": "Miami International Airport"
      }
    ]
  },
...
]
```

Finally, using the [shortestPath()](https://neo4j.com/docs/cypher-manual/current/appendix/tutorials/shortestpath-planning/)  function, you can find the shortest path between the airport Guillermo LeГіn Valencia (`'PPN'`) in Colombia and Newman Airport (`'ZNE'`) in Australia.

In [ ]:
query ="MATCH p=shortestPath((origin:Airport {code: 'PPN'})-[:Route*1..20]->(dest:Airport{code: 'ZNE'})) RETURN p"
records = execute_query(query)
print(records)

The previous query will return the shortest path assuming all routes are the same, but the air routes graph does contain the distance for each route as a property. So you can refer to the air routes graph as a weighted graph, where each edge (i.e. route) between nodes has a weight which is denoted by the distance of that route. To perform a more accurate search in a weighted graph you should use an algorithm such as the Dijkstra Shortest Path Algorithm. This is one of the procedures that has already been implemented in Neo4j and in most Graph Databases. This algorithm is out of the scope of this lab, but if you are interested, you can read about it [here](https://neo4j.com/docs/graph-data-science/current/algorithms/dijkstra-single-source/).

<a name='4'></a>
## 4 - Upload Files for Grading

Upload the notebook into S3 bucket for grading purposes.

*Note*: you may need to click **Save** button before the upload.

In [ ]:
# Retrieve the AWS account ID
result = subprocess.run(['aws', 'sts', 'get-caller-identity', '--query', 'Account', '--output', 'text'], capture_output=True, text=True)
AWS_ACCOUNT_ID = result.stdout.strip()

SUBMISSION_BUCKET = f"{LAB_PREFIX}-{AWS_ACCOUNT_ID}-us-east-1-submission"

!aws s3 cp ./C3_W1_Assignment.ipynb s3://$SUBMISSION_BUCKET/C3_W1_Assignment_Learner.ipynb

<a name='5'></a>
## 5 - Vector Search

*Note:* This section and Exercise 17 are optional (ungraded).

Vector search is a powerful technique in information retrieval, it relies on data representations as vectors in a high-dimensional space. These vectors capture the semantic relationships and similarities between the data points. By leveraging embeddings, which are dense vector representations learned from data, vector search enables efficient and accurate retrieval of relevant information. Neo4j can integrate vector search capabilities by storing embeddings as properties of graph nodes or relationships. This integration empowers Neo4j to perform advanced similarity searches, recommendation systems, and other Machine Learning tasks within the context of graph data structures, enhancing its utility in various domains such as recommendation engines, fraud detection, and knowledge graphs.

First, you already have some embeddings in a CSV file that is already hosted in the Neo4j server. To check the file, use the following query to get the first record:

In [ ]:
query ="""LOAD CSV WITH HEADERS FROM 'file:///air-routes-latest-nodes-with-embeddings.csv' AS row
RETURN row LIMIT 1"""
records = execute_query(query)
print(records)

Now you will proceed to load all the embeddings. You use `MATCH` to find the node and then add the embedding as a property using the `SET` keyword (this is an Update operation!). You can check the embedding file schema by using the following query:

In [ ]:
query ="""LOAD CSV WITH HEADERS FROM 'file:///air-routes-latest-nodes-with-embeddings.csv' AS row
WITH * WHERE row.label = "airport"
MATCH (a:Airport {id: toInteger(row.id)})
SET a.embedding = toFloatList(split(row.embedding,';'))
RETURN count(a);"""
result = execute_query(query)

<a name='ex17'></a>
### Exercise 17

1. Go to the Neo4j user interface that you opened in another window in section step [1.2.2.](#1.2). Copy the command from below into the Neo4j command line.
![image](./images/neo4j-cmd.png)

```cypher
CREATE VECTOR INDEX `air-route-embeddings` FOR (n: None) ON (n.None) OPTIONS {indexConfig: {
 `vector.dimensions`: 1536,
 `vector.similarity_function`: 'cosine'
}}
```

2. You'll need to complete the code by replacing all of the `None` placeholders. Remember, you are trying to create a vector index for the `Airport` nodes on the `embedding` property. You should use the `'cosine'` similarity function when setting up the configurations. You can read more about the vector index and similarity functions in the documentation [here](https://neo4j.com/docs/cypher-manual/current/indexes/semantic-indexes/vector-indexes/).

3. Once you have completed the code you can click run. Then wait until you see the message *Add 1 index, completed after x ms.*
![image](./images/neo4j-cmd-run.png)

Now that you have the embeddings and a vector index created on top of them, you can start doing a vector search on the graph. This is an example of a query using the vector index to find 5 nodes with similar embeddings and a score of the similarity between them; in this case you want to find airports similar to the one with code `'ANC'`.

In [ ]:
vector_query = """MATCH (a:Airport {code: 'ANC'}) 
CALL db.index.vector.queryNodes('air-route-embeddings', 5, a.embedding)
YIELD node AS similar_airport, score
MATCH (similar_airport)
RETURN similar_airport.code, similar_airport.country, similar_airport.desc, similar_airport.region, score"""
records = execute_query(vector_query)
print(records)

##### __Expected Output__ 

*Note*: Not all of the output is shown and the order of the records can be different.

```json
[
  {
    "similar_airport.code": "ANC",
    "similar_airport.country": "US",
    "similar_airport.desc": "Anchorage Ted Stevens",
    "similar_airport.region": "US-AK",
    "score": 1.0
  },
  {
    "similar_airport.code": "AUK",
    "similar_airport.country": "US",
    "similar_airport.desc": "Alakanuk Airport",
    "similar_airport.region": "US-AK",
    "score": 0.9300153255462646
  },
...
]
```

As you can appreciate the top result is the same node you were using to search for similar ones, the airport is Anchorage Ted Stevens and it is located in Alaska. The next 4 results are airports with similar embeddings, in this case, they are also airports in Alaska; as the airports share similar properties is no wonder why they are the most similar to the one in Anchorage.

Now run a vector search to find 10 airports similar to LaGuardia Airport in New York, retrieve their code, country, description and region.

In [ ]:
vector_query = """MATCH (a:Airport {code: 'LGA'}) 
CALL db.index.vector.queryNodes('air-route-embeddings', 10, a.embedding)
YIELD node AS similar_airport, score
MATCH (similar_airport)
RETURN similar_airport.code, similar_airport.country, similar_airport.desc, similar_airport.region, score"""
records = execute_query(vector_query)
print(records)

##### __Expected Output__ 

*Note*: Not all of the output is shown and the order of the records can be different.

```json
[
  {
    "similar_airport.code": "LGA",
    "similar_airport.country": "US",
    "similar_airport.desc": "New York La Guardia",
    "similar_airport.region": "US-NY",
    "score": 0.9999998807907104
  },
  {
    "similar_airport.code": "ISP",
    "similar_airport.country": "US",
    "similar_airport.desc": "Long Island Mac Arthur Airport",
    "similar_airport.region": "US-NY",
    "score": 0.8915630578994751
  },
...
]
```

This lab has demonstrated the significant advantages of leveraging graph databases and vector search techniques. By representing data as interconnected nodes and relationships, graph databases inherently capture complex relationships and dependencies, enabling more nuanced and insightful analysis compared to traditional relational databases. When combined with vector search capabilities, you can also extend the functionality of graph representation to incorporate semantic similarity searches, recommendation systems, and other advanced analytics tasks. You can find different Graph Database alternatives in the market apart from Neo4j. For example in AWS, Amazon Neptune is a fully managed graph database service that enables building graph-based applications in the cloud. One of Neptune's advantages is that it incorporates a variety of well-known graph query languages such as OpenCypher.

In case if you want to go back to the exercises before this optional section, you may want to cleanup the embeddings. To do that, go to the Neo4j user interface that you opened in another window in section step [1.2.2.](#1.2). Copy the command from below into the Neo4j command line to remove the index.

```cypher
DROP INDEX `air-route-embeddings` 
```

Then run the following cell to perform the cleanup of the embeddings:

In [ ]:
cleanup_query = """MATCH (a:Airport)
WHERE a.embedding IS NOT NULL
REMOVE a.embedding
RETURN count(a)"""
records = execute_query(cleanup_query)
print(records)